In [1]:
import random
import numpy as np


queries_cuis = np.array(['cui_1', 'cui_2', 'cui_3', 'cui_4', 
'cui_5', 'cui_6', 'cui_7', 'cui_8', 'cui_9', 'cui_10'])

dictionary_cuis = np.array(['cui_1', 'cui_2', 'cui_3', 'cui_4', 
'cui_5', 'cui_6', 'cui_7', 'cui_8', 'cui_9', 'cui_10', 
'cui_11', 'cui_12', 'cui_13', 'cui_14', 'cui_15', 
'cui_16', 'cui_17', 'cui_18', 'cui_19', 'cui_20',
'cui_1',  'cui_22'
])

candidates_idxs = []
for query in queries_cuis:
    random_cands = random.sample(range(len(dictionary_cuis)), k=5)
    candidates_idxs.append(random_cands)

candidates_idxs = np.array(candidates_idxs)





In [2]:
candidates_idxs = np.array([[20, 21, 10, 0,  1],
       [18,  6,  1, 21,  9],
       [19,  9,  6, 14, 13],
       [ 2, 11, 20, 21, 16],
       [14,  9,  4, 13,  3],
       [ 0,  2,  9, 15, 12],
       [ 6, 13, 15, 18,  3],
       [21, 14,  0,  3, 16],
       [10, 19,  1,  3, 20],
       [20, 10,  9, 17,  0]])


In [3]:
from torch.utils.data import Dataset, DataLoader 


class MyDataset(Dataset):
    def __init__(self, queries_cuis, candidates_idxs, dictionary_cuis):
        self.queries_cuis = queries_cuis
        self.candidates_idxs = candidates_idxs
        self.dictionary_cuis = dictionary_cuis

    def __len__(self):
        return len(self.queries_cuis)

    def __getitem__(self, idx):
        query_cui = self.queries_cuis[idx]
        candidate_idxs = self.candidates_idxs[idx]
        query_candidates_cuis = np.array(self.dictionary_cuis)[candidate_idxs]
        labels = (query_candidates_cuis == query_cui).astype(np.float32)
        return labels, query_candidates_cuis.tolist()


ds = MyDataset( queries_cuis, candidates_idxs, dictionary_cuis)
dl = DataLoader(ds, batch_size=25, shuffle=False)




In [5]:

import torch 
batch_size = 10
topk = 5
for dl_item in dl:
    labels , _ = dl_item
    break

scores = torch.rand((batch_size, topk), dtype=torch.float32)
scores_detached = scores
labels_detached = labels

num_pos = labels_detached.sum(dim=1)



num_pos = labels_detached.sum(dim=1)
num_neg = (1.0 - labels_detached).sum(dim=1)

num_pos_safe = torch.clamp(num_pos, min=1.0)
num_neg_safe = torch.clamp(num_neg, min=1.0)

pos_scores_sum = (scores_detached * labels_detached).sum(dim=1)
avg_pos_score = pos_scores_sum / num_pos_safe

neg_scores_sum = (scores_detached * (1.0 - labels_detached)).sum(dim=1)
avg_neg_score = neg_scores_sum / num_neg_safe

batch_margin = (avg_pos_score - avg_neg_score).mean().item()
batch_margin


-0.2356739342212677

In [9]:
scores= scores 
targets = labels 

targets_float = torch.zeros_like(scores)
valid_mask = (targets >= 0) & (targets < scores.size(1))
print(valid_mask)
targets_float.scatter_(1, targets[valid_mask].unsqueeze(1), 1.0)

num_pos = targets_float.sum(dim=1)
num_neg = (1.0 - targets_float).sum(dim=1)

# Avoid division by zero
num_pos_safe = num_pos.clamp(min=1.0)
num_neg_safe = num_neg.clamp(min=1.0)

pos_scores_sum = (scores * targets_float).sum(dim=1)
neg_scores_sum = (scores * (1.0 - targets_float)).sum(dim=1)

avg_pos_score = pos_scores_sum / num_pos_safe
avg_neg_score = neg_scores_sum / num_neg_safe

# Margin: avg_pos - avg_neg
# If a query has no positives, avg_pos_score is 0.
margin = (avg_pos_score - avg_neg_score).mean().item()
margin

tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])


RuntimeError: scatter(): Expected dtype int32/int64 for index

In [22]:

for i, dl_item in enumerate(dl):
    labels, queries_cuis = dl_item
    print(labels)
    print(queries_cuis)
    
    break

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 1.]])
[('cui_1', 'cui_19', 'cui_20', 'cui_3', 'cui_15'), ('cui_22', 'cui_7', 'cui_10', 'cui_12', 'cui_10'), ('cui_11', 'cui_2', 'cui_7', 'cui_1', 'cui_5'), ('cui_1', 'cui_22', 'cui_15', 'cui_22', 'cui_14'), ('cui_2', 'cui_10', 'cui_14', 'cui_17', 'cui_4')]
